In [1]:
import simpy
import numpy as np

In [2]:
# 시뮬레이션 파라미터
num_vehicles = 10
vehicle_speed = 140 / 3.6  # km/h를 m/s로 변환
resource_pool_size = 10  # 사용 가능한 서브프레임 수
transmission_power = 23  # dBm
cam_size = 190  # bytes
cam_periodicity = 0.1  # seconds
subframe_duration = 0.001  # seconds
doca_length = 500  # meters
control_period_duration = resource_pool_size * subframe_duration  # 제어 주기의 길이
env = simpy.Environment()

In [3]:
# 전송 및 수신 카운터
total_packets_sent = 0
successful_packets_received = 0

In [4]:
# 스케줄러 클래스
class Scheduler:
    def __init__(self, env, num_subframes):
        self.env = env
        self.num_subframes = num_subframes
        self.subframe_usage = [False] * num_subframes  # 각 서브프레임의 사용 여부

    def allocate_subframe(self):
        for i in range(self.num_subframes):
            if not self.subframe_usage[i]:
                self.subframe_usage[i] = True
                return i
        return None  # 모든 서브프레임이 사용중이라면 None 반환

    def release_subframe(self, subframe):
        self.subframe_usage[subframe] = False  # 서브프레임 사용 종료

In [5]:
# 차량 모델
class Vehicle:
    def __init__(self, env, id, speed, scheduler):
        self.env = env
        self.id = id
        self.speed = speed
        self.scheduler = scheduler
        self.action = env.process(self.run())

    def run(self):
        global total_packets_sent, successful_packets_received
        print("processing...")
        while True:
            # CAM 생성 및 스케줄러로부터 서브프레임 할당 받기
            subframe = self.scheduler.allocate_subframe()
            if subframe is not None:  # 서브프레임 할당 성공
                total_packets_sent += 1
                yield self.env.timeout(cam_periodicity)  # CAM 주기 대기
                # 패킷 전송 (서브프레임 지속 시간 동안)
                yield self.env.timeout(subframe_duration)
                successful_packets_received += 1  # 성공적인 패킷 수신 가정
                self.scheduler.release_subframe(subframe)  # 서브프레임 해제
                print(f"Time {self.env.now}: Vehicle {self.id} transmitted a CAM on subframe {subframe}.")
            else:  # 서브프레임 할당 실패
                print("fail allocation subFrame")
                yield self.env.timeout(cam_periodicity)  # 다음 주기까지 대기

In [6]:
# 스케줄러 인스턴스 생성
scheduler = Scheduler(env, resource_pool_size)

In [7]:
# 차량 생성 및 시뮬레이션 시작
vehicles = [Vehicle(env, i, vehicle_speed, scheduler) for i in range(num_vehicles)]

number_of_control_periods = 100 #원하는 제어 주기 수

total_simulation_time = control_period_duration * number_of_control_periods

env.run(until=total_simulation_time)  # 하나의 제어 주기 동안 실행

processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
processing...
Time 0.101: Vehicle 0 transmitted a CAM on subframe 0.
Time 0.101: Vehicle 1 transmitted a CAM on subframe 1.
Time 0.101: Vehicle 2 transmitted a CAM on subframe 2.
Time 0.101: Vehicle 3 transmitted a CAM on subframe 3.
Time 0.101: Vehicle 4 transmitted a CAM on subframe 4.
Time 0.101: Vehicle 5 transmitted a CAM on subframe 5.
Time 0.101: Vehicle 6 transmitted a CAM on subframe 6.
Time 0.101: Vehicle 7 transmitted a CAM on subframe 7.
Time 0.101: Vehicle 8 transmitted a CAM on subframe 8.
Time 0.101: Vehicle 9 transmitted a CAM on subframe 9.
Time 0.202: Vehicle 0 transmitted a CAM on subframe 0.
Time 0.202: Vehicle 1 transmitted a CAM on subframe 1.
Time 0.202: Vehicle 2 transmitted a CAM on subframe 2.
Time 0.202: Vehicle 3 transmitted a CAM on subframe 3.
Time 0.202: Vehicle 4 transmitted a CAM on subframe 4.
Time 0.202: Vehicle 5 transmitted a

In [9]:
# PRR 계산
prr = successful_packets_received / total_packets_sent if total_packets_sent > 0 else 0
print(f"PRR: {prr:.2f}")

PRR: 0.90
